In [1]:
pip install transformers datasets accelerate evaluate pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 71.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [2]:
pip install --upgrade transformers


In [3]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=efa09d300a60c219e8f0d1d7c3a7bf2fff37374960b0403533f2b8a913ad9c5b
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [4]:
pip install -U datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 15.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


## Load Dataset


In [5]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch

dataset = load_dataset("PRAli22/Arabic_dialects_to_MSA")



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

arabic_dialects_to_msa.csv:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/145541 [00:00<?, ? examples/s]

# Load model

In [6]:
model_name = "Murhaf/AraT5-MSAizer"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json:   0%|          | 0.00/835 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

#tokenizer


In [7]:
def preprocess_function(examples):
    inputs = ["translate dialect to MSA: " + ex for ex in examples["source"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["target"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/145541 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


# Evaluating model before trainin


In [19]:
def generate_msa(text):
    input_text = "translate dialect to MSA: " + text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True)

    outputs = model.generate(
        **inputs,
        max_length=128,
        num_beams=4,
        early_stopping=True
    )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

sample = dataset["train"].select(range(200))
import evaluate

bleu = evaluate.load("bleu")

predictions = [generate_msa(item["source"]) for item in sample]
references = [item["target"] for item in sample]

bleu_result = bleu.compute(predictions=predictions, references=references)

print("BLEU score:", bleu_result["bleu"])




BLEU score: 0.13025522968137948


In [8]:
from transformers import Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import torch

training_args = Seq2SeqTrainingArguments(
    output_dir="./finetuned-dialect-to-msa",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    eval_strategy="epoch",
    save_strategy="epoch",
    predict_with_generate=True,
    logging_dir="./logs",
    logging_steps=50,
    fp16=False,
    report_to=[],
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Train

In [9]:
from sklearn.model_selection import train_test_split
from datasets import Dataset
from transformers import Seq2SeqTrainer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments

train_data = tokenized_dataset["train"]
train_indices, val_indices = train_test_split(range(len(train_data)), test_size=0.1, random_state=42)

train_dataset = train_data.select(train_indices)
eval_dataset = train_data.select(val_indices)




data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset.select(range(5000)),  # استخدم عينة أولاً
    eval_dataset=eval_dataset.select(range(500)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/tmp/ipython-input-9-3226095991.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,0.172600,0.153428
2,0.149800,0.149936
3,0.173200,0.147696


TrainOutput(global_step=7500, training_loss=0.5642084217071534, metrics={'train_runtime': 2569.9296, 'train_samples_per_second': 5.837, 'train_steps_per_second': 2.918, 'total_flos': 3258641940480000.0, 'train_loss': 0.5642084217071534, 'epoch': 3.0})

In [10]:
def translate(text):
    input_text = "translate dialect to MSA: " + text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

print(translate("انا ماني فاضي لك"))
print(translate("شنو هالحظ؟!"))
print(translate("وش تبيني اسوي الحين؟"))


أنا لست مشغول بك
ما هذا الحظ ؟
ماذا تريد أن أفعل الآن ؟


# Evaluating model after trainin


In [14]:
import evaluate

bleu = evaluate.load("bleu")

eval_data = eval_dataset.select(range(200))
inputs = [ex["source"] for ex in eval_data]
references = [ex["target"] for ex in eval_data]


predictions = []
for text in inputs:
    input_text = "translate dialect to MSA: " + text
    input_tokens = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    output = model.generate(**input_tokens, max_new_tokens=100)
    decoded = tokenizer.decode(output[0], skip_special_tokens=True)
    predictions.append(decoded)

bleu_result = bleu.compute(predictions=predictions, references=[[r] for r in references])


print(f"\n📊 BLEU Score: {bleu_result['bleu']:.4f}")


📊 BLEU Score: 0.2243


In [15]:
model.save_pretrained("./finetuned-mt5-arabic-dialect-to-msa")
tokenizer.save_pretrained("./finetuned-mt5-arabic-dialect-to-msa")


('./finetuned-mt5-arabic-dialect-to-msa/tokenizer_config.json',
 './finetuned-mt5-arabic-dialect-to-msa/special_tokens_map.json',
 './finetuned-mt5-arabic-dialect-to-msa/spiece.model',
 './finetuned-mt5-arabic-dialect-to-msa/added_tokens.json',
 './finetuned-mt5-arabic-dialect-to-msa/tokenizer.json')

In [16]:
!pip install gradio transformers

import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_path = "./finetuned-mt5-arabic-dialect-to-msa"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

def translate(text):
    if not text.strip():
        return "من فضلك أدخل جملة."
    input_text = "translate dialect to MSA: " + text
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, padding=True).to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=100)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

iface = gr.Interface(fn=translate, inputs="text", outputs="text", title="محول العامية إلى الفصحى")
iface.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0d01fea63cd72d74cd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [20]:
!zip -r /content/finetuned_model.zip ./finetuned-mt5-arabic-dialect-to-msa


  adding: finetuned-mt5-arabic-dialect-to-msa/ (stored 0%)
  adding: finetuned-mt5-arabic-dialect-to-msa/model.safetensors (deflated 11%)
  adding: finetuned-mt5-arabic-dialect-to-msa/spiece.model (deflated 55%)
  adding: finetuned-mt5-arabic-dialect-to-msa/special_tokens_map.json (deflated 85%)
  adding: finetuned-mt5-arabic-dialect-to-msa/config.json (deflated 49%)
  adding: finetuned-mt5-arabic-dialect-to-msa/tokenizer.json (deflated 84%)
  adding: finetuned-mt5-arabic-dialect-to-msa/tokenizer_config.json (deflated 94%)
  adding: finetuned-mt5-arabic-dialect-to-msa/generation_config.json (deflated 29%)


In [21]:
from google.colab import files
files.download('/content/finetuned_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>